# Modelo Privatizado con epsilon 1

Utilizando la base privatizada con laplace y OUE con un epsilon tomando valor igual a  1 (su obtención se da con el archivo 2_Analisis_Exploratorio) se procede a crear modelo con el fin de comparar.

In [48]:
import pickle
import pandas as pd

archivo_pickle = 'base_epsilon_1.0.pkl'


with open(archivo_pickle, 'rb') as file:
    df = pickle.load(file)


Como se hizo en el modelo sin privatizar se trabajara el con personas que tengan una enfermedad, excuyendo asi a los sanos

In [49]:
df = df[df['Enfermedad'] != 'Sin Enfermedad']

Se vio en el procedimiento de la privatización que juntamos todas las enfermedades separando por , pero para el modelamiento se necesita volver a ampliar la base, dummificando la enfermedad.

In [50]:
df['Enfermedad'] = df['Enfermedad'].str.split(',')

# Crear variables dummies
enfermedades_dummies = df['Enfermedad'].str.join('|').str.get_dummies()

In [52]:
df = pd.concat([df.drop('Enfermedad', axis=1), enfermedades_dummies], axis=1)

Se eliminan las filas donde no se nos aporta la información ya que vienen en 0 y las columnas que no son relevantes para el modelo.

In [58]:
# Eliminar columnas donde todas las entradas son cero
df = df.loc[:, (df != 0).any(axis=0)]
df['DIAS_COMP'] = df['DIAS_COMP'].astype(float)
df.drop(columns='MUNI', inplace=True)

Se seleccionan varias variables categóricas, específicamente 'EPS', 'SEXO', 'DPTO', 'V8', 'Grupo_Edad', 'Zona', para aplicarles el proceso de One-Hot Encoding, con el fin de transformarlas en formatos adecuados, garantizando que el dataset solo contenga variables numéricas. Finalmente, se define 'Valor_Neto' como la variable objetivo del estudio y se procede a dividir el dataset en conjuntos de entrenamiento y prueba, asignando el 25% de los datos al conjunto de prueba, utilizando un estado aleatorio de 42 para asegurar la reproducibilidad de los datos.

In [59]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

df = df.dropna()

# Aplicar One-Hot Encoding a las variables categóricas
categorical_columns = ['EPS', 'SEXO', 'DPTO', 'V8', 'Grupo_Edad', 'Zona']
encoder = OneHotEncoder()
encoded_categoricals = encoder.fit_transform(df[categorical_columns])
encoded_df = pd.DataFrame(encoded_categoricals.toarray(), columns=encoder.get_feature_names_out(categorical_columns), index=df.index)

# Concatenar los datos codificados con el resto del dataframe
df = pd.concat([df.drop(categorical_columns, axis=1), encoded_df], axis=1)

# Dividir el conjunto de datos en entrenamiento y prueba
X = df.drop('Valor_Neto', axis=1)
y = df['Valor_Neto']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, shuffle=True)

Se lleva a cabo la inicialización, entrenamiento y evaluación de múltiples modelos de regresión, como Regresión Lineal, Árbol de Decisión, Bosque Aleatorio y XGBoost, con el objetivo de establecer una línea base de métricas de desempeño. Al utilizar varios modelos, se facilita la comparación directa de su rendimiento en el mismo conjunto de datos, lo que permite identificar cuál modelo ofrece la mejor precisión y generalización. Para cada modelo, se calculan y registran métricas clave como el coeficiente de determinación (R2), el error absoluto medio (MAE) y el error cuadrático medio (MSE) tanto en el conjunto de entrenamiento como en el de prueba.

In [63]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Inicializar los modelos
models = {
    'Linear Regression': LinearRegression(),
    'Decision Tree': DecisionTreeRegressor(),
    'Random Forest': RandomForestRegressor(),
    'XGBoost': XGBRegressor(verbosity=0)
}

# Entrenar y evaluar los modelos
results = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    results[name] = {
        'train': {
            'R2': r2_score(y_train, y_train_pred),
            'MAE': mean_absolute_error(y_train, y_train_pred),
            'MSE': mean_squared_error(y_train, y_train_pred)
        },
        'test': {
            'R2': r2_score(y_test, y_test_pred),
            'MAE': mean_absolute_error(y_test, y_test_pred),
            'MSE': mean_squared_error(y_test, y_test_pred)
        }
    }

# Imprimir resultados
for model_name, metrics in results.items():
    print(f"Modelo: {model_name}")
    for dataset_type, values in metrics.items():
        print(f"\t{dataset_type} Metrics:")
        for metric_name, metric_value in values.items():
            print(f"\t\t{metric_name}: {metric_value:.4f}")

Modelo: Linear Regression
	train Metrics:
		R2: 0.0427
		MAE: 145286.1754
		MSE: 231967534040.7825
	test Metrics:
		R2: -241445910.0610
		MAE: 20848791.0509
		MSE: 68094847308776226816.0000
Modelo: Decision Tree
	train Metrics:
		R2: 1.0000
		MAE: 0.0000
		MSE: 0.0000
	test Metrics:
		R2: -0.2354
		MAE: 130419.1630
		MSE: 348415973265.7783
Modelo: Random Forest
	train Metrics:
		R2: 0.8629
		MAE: 38192.4211
		MSE: 33216458831.3307
	test Metrics:
		R2: 0.1384
		MAE: 102000.2117
		MSE: 242986601574.2516
Modelo: XGBoost
	train Metrics:
		R2: 0.5690
		MAE: 90749.8897
		MSE: 104435125033.2767
	test Metrics:
		R2: -0.0108
		MAE: 97372.7589
		MSE: 285067909227.2698


Al evaluar los modelos con datos privatizados, tanto la Regresión Lineal como el Árbol de Decisión enfrentan problemas significativos, con el Árbol mostrando un claro overfitting. Dado esto, XGBoost es  la opción más importante debido a su flexibilidad y robustez.

Usando Optuna para afinar XGBoost, se busca mejorar significativamente su rendimiento, equilibrando la precisión en el entrenamiento con la capacidad de generalizar bien en nuevos datos. Esta búsqueda de hiperparámetros no solo mejora la precisión del modelo, sino que también adapta el modelo proporcionando un balance entre ajuste y generalización.

In [65]:
import optuna
from xgboost import XGBRegressor
from sklearn.metrics import r2_score

def objective(trial):
    param = {
        'verbosity': 0,
        'objective': 'reg:squarederror',
        'tree_method': 'exact',  # para modelos más pequeños, 'exact' es adecuado
        'lambda': trial.suggest_loguniform('lambda', 1e-8, 1.0),
        'alpha': trial.suggest_loguniform('alpha', 1e-8, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 1.0),
        'subsample': trial.suggest_float('subsample', 0.1, 1.0),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 20),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 300),
    }
    
    model = XGBRegressor(**param)
    model.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=False, early_stopping_rounds=50)
    y_pred = model.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    return r2

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=40)

best_params = study.best_params
print("Mejores parámetros:", best_params)

# Entrenar el modelo con los mejores parámetros
optimized_model = XGBRegressor(**best_params)
optimized_model.fit(X_train, y_train)

# Evaluación del modelo
y_train_pred = optimized_model.predict(X_train)
y_test_pred = optimized_model.predict(X_test)
train_r2 = r2_score(y_train, y_train_pred)
test_r2 = r2_score(y_test, y_test_pred)

print(f"R2 Train: {train_r2:.4f}")
print(f"R2 Test: {test_r2:.4f}")


[I 2024-05-18 11:41:05,717] A new study created in memory with name: no-name-21112354-a8d6-4d79-a7bf-e99cf8840225
/var/folders/tv/6r5469xx31vbksvh_943nnr1bcv0mh/T/ipykernel_48795/1286862207.py:10: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'lambda': trial.suggest_loguniform('lambda', 1e-8, 1.0),
/var/folders/tv/6r5469xx31vbksvh_943nnr1bcv0mh/T/ipykernel_48795/1286862207.py:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'alpha': trial.suggest_loguniform('alpha', 1e-8, 1.0),
/opt/anaconda3/lib/python3.11/site-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stop

Mejores parámetros: {'lambda': 0.00024701594556858855, 'alpha': 0.007821328116371975, 'colsample_bytree': 0.8762333637859853, 'subsample': 0.20152504325518536, 'learning_rate': 0.07130613535664129, 'n_estimators': 779, 'max_depth': 13, 'min_child_weight': 39}
R2 Train: 0.3011
R2 Test: 0.1271


La optimización mediante Optuna ha ajustado los parámetros clave de XGBoost, llevando el R2 en el entrenamiento a 0.3011 y en la prueba a 0.1271. Aunque estos números son una mejora, todavía hay una brecha entre cómo el modelo se comporta en los datos de entrenamiento y prueba, lo que indica que podemos seguir ajustando para alcanzar un mejor equilibrio y precisión. Esta búsqueda de hiperparámetros es un paso positivo hacia hacer el modelo más efectivo en escenarios reales con datos privatizados.

In [ ]:
with open('model_priv1.pkl', 'wb') as file:
    pickle.dump(optimized_model, file)